# Qlora for Llama3. runs with "qlora3" kernel
- Source: https://medium.com/@avishekpaul31/fine-tuning-llama-3-8b-instruct-qlora-using-low-cost-resources-89075e0dfa04
- SOurce: https://github.com/AvisP/LM_Finetune/blob/main/llama-3-finetune-qlora.ipynb

In [ ]:
# workaround for packaging fehler
pip install setuptools==69.5.1

In [ ]:
!pip install torch==2.0.1
!pip install bitsandbytes
!pip install -U transformers[torch] datasets
!pip install -q bitsandbytes trl peft accelerate
!pip install flash-attn --no-build-isolation
!pip install transformers==4.40.2 # Es geht nur genau diese Version!!
!pip install trl
!pip install autoawq
!pip install huggingface_hub

In [2]:
from huggingface_hub import notebook_login
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

notebook_login()


### Set Model, Tokenizer & Output

In [ ]:
!conda list | grep trl

In [ ]:
!pip install sentencepiece
!pip install mistral_inference


In [ ]:
model_id = "/home/thsch026/masterarbeit/models/mistral/mistral-7B-v0.1"
model_id = "mistralai/Mistral-7B-v0.3"

# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Resulting Model Storage
trained_model_id = "Mistral-7b_full"
output_dir = '/home/thsch026/masterarbeit/models/generated/qlora/' + trained_model_id

## Load Dataset and trim for training

In [ ]:
haha = "/home/thsch026/masterarbeit/models/mistral/mistral-7B-v0.3"
tokenizer.save_pretrained(haha)
model.save_pretrained(haha)

In [3]:
from datasets import load_dataset

# based on config
raw_datasets = load_dataset("HuggingFaceH4/ultrachat_200k")

In [4]:
from datasets import DatasetDict

# remove this when done debugging to include whole dataset
indices = range(0,10000)

dataset_dict = {"train": raw_datasets["train_sft"].select(indices),
                "test": raw_datasets["test_sft"].select(indices)}

#dataset_dict = {"train": raw_datasets["train_sft"],
  #              "test": raw_datasets["test_sft"]}

raw_datasets = DatasetDict(dataset_dict)
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 10000
    })
})

In [ ]:
# raw_datasets["train"][:2]['messages']

In [5]:
example = raw_datasets["train"][0]
messages = example["messages"]
for message in messages:
  role = message["role"]
  content = message["content"]
  print('{0:20}:  {1}'.format(role, content))

user                :  These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?
On your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!
Your Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.
Does this feature apply to all sections of the theme or just specific ones as listed in the text material?
assistant           :  This feature only applies to Collection pages and Featured Collections sections of the section-based themes listed in the text material.
user                :  Can you guide me through the process of enabling the secondary image hover feature on my Collection pages and Featured Collections sections?
assistant           :  Sure, here are the steps to enable the secondary 

In [6]:
tokenizer.eos_token_id

128009

In [7]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [8]:
import re
import random
from multiprocessing import cpu_count

def apply_chat_template(example, tokenizer):
    messages = example["messages"]
    # We add an empty system message if there is none
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})
    example["text"] = tokenizer.apply_chat_template(messages, tokenize=False)

    return example

In [9]:
column_names = list(raw_datasets["train"].features)
raw_datasets = raw_datasets.map(apply_chat_template,
                                num_proc=cpu_count(),
                                fn_kwargs={"tokenizer": tokenizer},
                                remove_columns=column_names,
                                desc="Applying chat template",)

Applying chat template (num_proc=128):   0%|          | 0/10000 [00:00<?, ? examples/s]

Applying chat template (num_proc=128):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [10]:
# set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

# Set reasonable default for models without max length
if tokenizer.model_max_length > 100_000:
  tokenizer.model_max_length = 2048

In [ ]:
# create the splits
train_dataset = raw_datasets["train"]
eval_dataset = raw_datasets["test"]

for index in random.sample(range(len(raw_datasets["train"])), 3):
  print(f"Sample {index} of the processed training set:\n\n{raw_datasets['train'][index]['text']}")
  print("#####################################")

In [12]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)
# https://github.com/Lightning-AI/litgpt/issues/327

outputs = model.generate(
    input_ids,
    max_new_tokens=128,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

response = outputs[0][input_ids.shape[-1]:]

print(tokenizer.decode(response, skip_special_tokens=True))

/home/thsch026/my-envs/qlora3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/home/thsch026/my-envs/qlora3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Arrrr, me hearty! Me name be Captain Chat, the scurviest pirate chatbot to ever sail the Seven Seas o' the Interwebs! Me and me trusty crew o' code have been plunderin' the riches o' knowledge and bringin' 'em back to ye, me matey! So hoist the sails and set course fer a swashbucklin' good time, savvy?


## Prepare Training

In [13]:
from trl import SFTTrainer
from peft import LoraConfig
from transformers import TrainingArguments
from transformers import BitsAndBytesConfig

In [14]:

# For 8 bit quantization
quantization_config = BitsAndBytesConfig(load_in_8bit=True,
                                        llm_int8_threshold=200.0)

## For 4 bit quantization
#quantization_config = BitsAndBytesConfig(
 #           load_in_4bit=True,
  #          bnb_4bit_use_double_quant=True,
   #         bnb_4bit_quant_type="nf4",
    #        bnb_4bit_compute_dtype=torch.bfloat16,)

#model = AutoModelForCausalLM.from_pretrained(model_id, 
 #                                            quantization_config=quantization_config,
  #                                      device_map="auto")

In [15]:
model_kwargs = dict(
#     attn_implementation=False,#"flash_attention_2", # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
    torch_dtype="auto",
    use_cache=False, # set to False as we're going to use gradient checkpointing
    device_map="auto",
    quantization_config=quantization_config,
)

In [16]:

# based on config
training_args = TrainingArguments(
    fp16=False, # specify bf16=True instead when training on GPUs that support bf16 else fp16
    bf16=False,
    do_eval=True,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    learning_rate=2.0e-05,
    log_level="info",
    logging_steps=5,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    max_steps=-1,
    num_train_epochs=1,
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_eval_batch_size=1, # originally set to 8
    per_device_train_batch_size=1, # originally set to 8
    push_to_hub=True,
    hub_model_id=trained_model_id,
    # hub_strategy="every_save",
    # report_to="tensorboard",
    report_to="none",  # for skipping wandb logging
    save_strategy="no",
    save_total_limit=None,
    seed=42,
)

# based on config
peft_config = LoraConfig(
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

trainer = SFTTrainer(
        model=model_id,
        model_init_kwargs=model_kwargs,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        dataset_text_field="text",
        tokenizer=tokenizer,
        packing=True,
        peft_config=peft_config,
        max_seq_length=tokenizer.model_max_length,
    )

# To clear out cache for unsuccessful run
torch.cuda.empty_cache()

/home/thsch026/my-envs/qlora3/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:166: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(
/home/thsch026/my-envs/qlora3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2991 > 2048). Running this sequence through the model will result in indexing errors


Generating train split: 0 examples [00:00, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


## Start Training

In [17]:
train_result = trainer.train()

***** Running training *****
  Num examples = 5,772
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 5,772
  Number of trainable parameters = 54,525,952
/home/thsch026/my-envs/qlora3/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss
1,1.135200,1.190444


***** Running Evaluation *****
  Num examples = 5756
  Batch size = 1


Training completed. Do not forget to share your model on huggingface.co/models =)




## Save results

In [ ]:
!conda list | grep torch

In [ ]:
!pip install torch==2.0.1

In [ ]:
!pip install trl

In [18]:
tokenizer.save_pretrained(output_dir)
model.save_pretrained(output_dir)

tokenizer config file saved in /home/thsch026/masterarbeit/models/generated/qlora/Llama-3-8B-qlora_full/tokenizer_config.json
Special tokens file saved in /home/thsch026/masterarbeit/models/generated/qlora/Llama-3-8B-qlora_full/special_tokens_map.json
Configuration saved in /home/thsch026/masterarbeit/models/generated/qlora/Llama-3-8B-qlora_full/config.json
Configuration saved in /home/thsch026/masterarbeit/models/generated/qlora/Llama-3-8B-qlora_full/generation_config.json
The model is bigger than the maximum size per checkpoint (5GB) and is going to be split in 4 checkpoint shards. You can find where each parameters has been saved in the index located at /home/thsch026/masterarbeit/models/generated/qlora/Llama-3-8B-qlora_full/model.safetensors.index.json.
